In [1]:
from itertools import combinations

import os
import sys
from pathlib import Path

sys.path.append(str((Path(os.path.abspath("")) / "..").resolve()))
from sudoku_variants import Sudoku, SudokuAI, SudokuConst
from sudoku_variants.rule import Orthogonal, SubBoard, Knight, King, Consecutive
from sudoku_variants.func import rules as R
from sudoku_variants.helper.profile import profile


def standard_rules():
    return [Orthogonal(), SubBoard()]


def variant_rules():
    return [Knight(), King(), Consecutive()]

### AI

In [12]:
@profile(10)
def run(rules):
    board = [[0 for _ in range(SudokuConst.NUM_COL)] for _ in range(SudokuConst.NUM_ROW)]
    sudoku = Sudoku(board, rules)
    SudokuAI.solve(sudoku)


for length in range(len(variant_rules()) + 1):
    for subset in combinations(variant_rules(), length):
        print(R.to_name(list(subset)))
        rules = standard_rules() + list(subset)
        run(rules)


Time: 0.0286s, 10x | 0.0029s/time
Knight
Time: 0.7580s, 10x | 0.0758s/time
King
Time: 0.1904s, 10x | 0.0190s/time
Consecutive
Time: 0.0318s, 10x | 0.0032s/time
Knight, King
Time: 1.9722s, 10x | 0.1972s/time
Knight, Consecutive
Time: 0.6011s, 10x | 0.0601s/time
King, Consecutive
Time: 0.1258s, 10x | 0.0126s/time
Knight, King, Consecutive
Time: 2.6119s, 10x | 0.2612s/time


In [2]:
@profile(10)
def run(rules, to_erase):
    SudokuAI.generate(rules, max_erased=to_erase)


for to_erase in [35, 40, 50, 60]:
    print(to_erase)
    rules = [Orthogonal(), SubBoard(), Knight(), King()]
    run(rules, to_erase)

35
Time: 41.8159s, 10x | 4.1816s/time
40


In [ ]:
@profile(1)
def run():
    sudoku = SudokuAI.generate([Orthogonal(), SubBoard(), Knight(), King(), Consecutive()],  max_erased=1)
    print(sudoku)


run()

[(0, 0)]
Applied rules: Orthogonal, SubBoard, Knight, King, Consecutive
+-------+-------+-------+
| * 9 5 | 6 8 2 | 4 1 3 |
| 4 1 3 | 7 9 5 | 6 8 2 |
| 6 8 2 | 4 1 3 | 7 9 5 |
+-------+-------+-------+
| 3 7 9 | 5 6 8 | 2 4 1 |
| 2 4 1 | 3 7 9 | 5 6 8 |
| 5 6 8 | 2 4 1 | 3 7 9 |
+-------+-------+-------+
| 1 3 7 | 9 5 6 | 8 2 4 |
| 8 2 4 | 1 3 7 | 9 5 6 |
| 9 5 6 | 8 2 4 | 1 3 7 |
+-------+-------+-------+
Time: 0.2140s, 1x | 0.2140s/time
